# Chapter 4: Random variables in SciPy

This programming assignment focuses on [SciPy](https://docs.scipy.org/doc/scipy/index.html), which is one of the main scientific computing libraries for Python. Beyond probability and statistics, it also provides functionality for many other areas of computational mathematics.

SciPy implements a [huge number](https://docs.scipy.org/doc/scipy/reference/stats.html#probability-distributions) of probability distributions, including all the ones that we will study in our class. It includes the mass and density functions of random variables, their cumulative distribution functions, and also their statistics like expectations, variances, and quantiles. We will explore all of these in this assignment. At the end, we will also see how to simulate datasets via sampling in SciPy, and we will make the connection with the empirical distributions studied in the [previous programming assignment](https://github.com/jmyers7/stats-book-materials/blob/main/programming-assignments/assignment_03.ipynb).

## Directions

1. The programming assignment is organized into sequences of short problems. You can see the structure of the programming assignment by opening the "Table of Contents" along the left side of the notebook (if you are using Google Colab or Jupyter Lab).

3. Each problem contains a blank cell containing the following comment: `# ENTER YOUR CODE IN THIS CELL`. Enter your code in these cells below the comment, being sure to not erase the comment. There are usually directions on the precise syntax that you will use to enter your solution properly. Please pay very careful attention to these directions.

4. Below most of the solution cells are "autograder" cells. Do not alter the autograder cells in any way.

5. Do not add any cells of your own to the notebook, or delete any existing cells (either code or markdown).

## Submission instructions

1. Once you have finished entering all your solutions, you will want to rerun all cells from scratch to ensure that everything works OK. To do this in Google Colab, click "Runtime -> Restart and run all" along the top of the notebook.

2. Now scroll back through your notebook and make sure that all code cells ran properly.

3. If everything looks OK, save your assignment and upload the `.ipynb` file at the provided link on the course <a href="https://github.com/jmyers7/stats-book-materials">GitHub repo</a>. Late submissions are not accepted.

4. You may submit multiple times, but I will only grade your last submission.

## Mass and density functions from scratch

## Description

Though the goal of this assignment is to learn how random variables are implemented in SciPy, let's start off by learning how to code mass and density functions from scratch. These skills will be very valuable later, when we code custom probabilistic models in Python in a [future chapter](https://mml.johnmyersmath.com/stats-book/chapters/models.html) of the book.

We first import SciPy under the alias `sp`, as well as the other standard and (by now) familiar libraries:

In [ ]:
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Then, let's begin with binomial random variables. We recall that their mass functions look like

$$
p(x;n,\theta) = \binom{n}{x} \theta^x ( 1-\theta)^{n-x},
$$

for $x=0,1,\ldots,n$. This may be implemented as follows:

In [ ]:
def p(x, n, theta):
  return sp.special.binom(n, x) * (theta ** x) * ((1 - theta) ** (n - x))

Note that I used the `binom` method from the `special` submodule in SciPy (see the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.binom.html#scipy.special.binom)) for the binomial coefficient. Notice also the proper usage of spacing and parentheses to increase readability and reduce ambiguity. Compare my code to this garbage code:

``` python
def p(x,n,theta):
  return sp.special(n,x)*theta**x*(1-theta)**(n-x)
```

Gross! As you progress in your journey and mature as a programmer, you want to pay attention to stylistic issues like these. So start practicing now! (Also, take a look at [this](https://peps.python.org/pep-0008/).)

Now, let's see if our mass function is correct. In the [section of the book](https://mml.johnmyersmath.com/stats-book/chapters/examples-of-rvs.html#binomial-distributions) on binomial random variables, there is a probability histogram of a random variable $X\sim Bin(8,0.75)$. Let's see if we can recreate it.

First, we define a NumPy array containing the support of the mass function and pass this into the PMF to obtain the corresponding probabilities:

In [ ]:
support = np.arange(0, 9)
probs = p(x=support, n=8, theta=0.75)
probs

Notice that `probs` is actually a NumPy array. But, if we want to use the same techniques that we used in the [previous programming assignment](https://github.com/jmyers7/stats-book-materials/blob/main/programming-assignments/assignment_03.ipynb) to produce a plot of the PMF, we need to create a Pandas Series:

In [ ]:
pmf = pd.Series(data=probs, index=support)
pmf

Notice that `pmf` is a Pandas Series whose data column contains the values of the mass function and whose indices contain the support.

Now, we use the same code from the [previous programming assignment](https://github.com/jmyers7/stats-book-materials/blob/main/programming-assignments/assignment_03.ipynb) to produce the histogram:

In [ ]:
pmf.plot(kind='bar', rot=0)
plt.xlabel('$x$')
plt.ylabel('probability')
plt.title('PMF of $X \sim Bin(8, 0.75)$')
plt.show()

That looks about right!

### Problem 1 --- PDFs of exponential variables from scratch

In the next code block, implement the density function for an [exponential random variable](https://mml.johnmyersmath.com/stats-book/chapters/examples-of-rvs.html#exponential-distributions) with call signature `f(x, l)` where `l` stands for $\lambda$. You might be tempted to use `lambda` instead of `l`, but you can't, because `lambda` is a reserved keyword in the Python language. Use `np.exp` for the exponential function.


In [ ]:
# ENTER YOUR CODE IN THIS CELL



In [ ]:
# Autograder cell. Do NOT alter or delete this cell.


Now, to make sure that your implementation is correct, produce the following plot using your density function `f`:

<center>
<img src="https://github.com/jmyers7/stats-book-materials/blob/482838c8622e479424b1b0d03b6a119c307f5817/img/exp-pdf.png?raw=true" width="600" align="center">
</center>

As always, I am looking for an _exact_ recreation, so take care to notice the axis labels and the title of the plot. If you don't know how to write "$X\sim Exp(3)$" in LaTeX, look at the plot in the previous section for hints. We produced plots like this in the [first programming assignment](https://github.com/jmyers7/stats-book-materials/blob/main/programming-assignments/assignment_01.ipynb). Look there for more hints.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



## Mass and density functions in SciPy

### Description

Having coded mass and density functions from scratch, we now begin learning how SciPy implements mass and density functions.

First up is the binomial random variable $X \sim Bin(8, 0.75)$ from the previous section. According to the the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html), its mass function is implemented in SciPy as follows:

In [ ]:
X = sp.stats.binom(n=8, p=0.75)

Notice that SciPy uses $p$ instead of $\theta$ for the second parameter. Also, if you look in the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html), you'll see that SciPy implements the mass function in slightly different notation compared to what we use in class:

$$
f(k) = \binom{n}{k} p^k (1-p)^{1-k}.
$$

In addition to using $p$ in place of $\theta$, notice that SciPy uses $f(k)$ rather than $p(x)$.

Let's evaluate $p(4)$ (in our notation) for our random variable $X$:

In [ ]:
X.pmf(4)

If you compare my function call to `pmf` to the call signature in the docs, you'll notice that I _only_ passed in `k=4` (as an unlabeled parameter) and left off the `n` and `p` parameters. I got away with this because I "froze" the `n` and `p` into `X` when I defined it above.

Let's recreate the probability histogram for $X$ from the previous section:

In [ ]:
support = np.arange(0, 9)
probs = X.pmf(support)
pmf = pd.Series(data=probs, index=support)
pmf.plot(kind='bar', rot=0)
plt.xlabel('$x$')
plt.ylabel('probability')
plt.title('PMF of $X \sim Bin(8, 0.75)$')
plt.show()

Looks the same, right?

Having looked at a discrete random variable, how about a continuous one? According to the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html), we would implement a random variable $Y\sim Gam(4, 3)$ as follows:

In [ ]:
Y = sp.stats.gamma(a=4, scale=1 / 3)

As you may read for yourself, the SciPy implementation of a random variable $Gam(\alpha,\beta)$ takes `a` as the $\alpha$ parameter and `scale` as the parameter $1/\beta$.

Let's plot the density function of $Y$ over the interval $[0,10]$. Hopefully by now you are starting to recognize familiar bits of code:

In [ ]:
grid = np.linspace(0, 10)
plt.plot(grid, Y.pdf(grid))
plt.xlabel('$y$')
plt.ylabel('probability density')
plt.title('PDF of $Y \sim Gam(4, 3)$')
plt.show()

### Problem 2 --- PMFs and PDFs of Poisson and normal variables

Now it's your turn. By using the code blocks above for hints and reading the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html), your goal is to first produce the following pair of histograms for Poisson random variables:

<center>
<img src="https://github.com/jmyers7/stats-book-materials/blob/6dd6d785d4fcc343ef8aa4d1f93b7414771ea2fb/img/poisson-pmf.png?raw=true" width="900" align="center">
</center>

To produce the side-by-side plots, you'll need to use the `subplots` method of `plt` as you learned back in the [first programming assignment](https://github.com/jmyers7/stats-book-materials/blob/main/programming-assignments/assignment_01.ipynb). Also:

* Pass in the parameters `figsize=(10,5)`, `sharey=True`, and `sharex=True` to `plt.subplots`.

* You will use the `plot` method of Pandas Series to produce the histograms, just like above. However, to place these histograms into the subplots, you'll need to pass the appropriate `axes` object produced by `subplots` into the `ax` parameter of `plot`; see [here](https://pandas.pydata.org/docs/reference/api/pandas.Series.plot.html).

* I am looking for an _exact_ recreation of the plots, so be sure to notice the range of values along the horizontal axes, the axis labels, and the plot titles.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Now, for the second part of the problem, your goal is to produce the following pair of plots for normal random variables: (See the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html)!)

<center>
<img src="https://github.com/jmyers7/stats-book-materials/blob/f07fda52a1bbbfd41389fa956116afe809955880/img/norm-pdf.png?raw=true" width="900" align="center">
</center>

* Again, pass in the parameters `figsize=(10,5)`, `sharey=True`, and `sharex=True` to `plt.subplots`.

* As always, I am looking for an _exact_ recreation of the plots. Pay attention to the details!

In [ ]:
# ENTER YOUR CODE IN THIS CELL



## Cumulative distribution functions and quantiles

### Description

SciPy also implements the `cdf` and `ppf` methods, which give the cumulative distribution functions and quantile functions for random variables (`ppf` stands for _percent point function_, which is what SciPy calls the quantile function).

They work exactly how you would expect. For example:

In [ ]:
Z = sp.stats.norm(loc=0, scale=1)
Z.cdf(3)

This code block returns $\Phi(3)$, where we recall $\Phi(z)$ stands for the CDF of a standard normal variable $Z$. For quantiles:

In [ ]:
Z.ppf(0.5)

This code block returns $Q(0.5)$, or the median of $Z$. But we already know that the median of a standard normal variable $Z$ is 0, so this answer isn't surprising. How about the $0.25$-quatile?

In [ ]:
Z.ppf(0.25)

Remember, the quantile function and CDF are supposed to be inverses, so if I put this answer back into the CDF, I should get back to where I started:

In [ ]:
Z.cdf(Z.ppf(0.25))

### Problem 3 --- CDFs and quantiles of normal variables

Your first goal in this problem is to produce this plot of the CDF and PDF of a random variable $X \sim N(1, 2^2)$:

<center>
<img src="https://github.com/jmyers7/stats-book-materials/blob/ec9cd6ffad8c1614e73230fcc511ed1dcbe82bd9/img/norm-pdf-cdf.png?raw=true" width="600" align="center">
</center>

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Now, compute the $0.75$-quantile of the normal variable $X \sim N(1, 2^2)$ in the previous code block. Save your answer into the variable `quantile`:

In [ ]:
# ENTER YOUR CODE IN THIS CELL



In [ ]:
# Autograder cell. Do NOT alter or delete this cell.


Using your $0.75$-quantile, you will produce the following plot:

<center>
<img src="https://github.com/jmyers7/stats-book-materials/blob/27731f92ad6f13d6ed9ec0fa3fb10ab0ad456aec/img/norm-quantile.png?raw=true" width="600" align="center">
</center>

The $0.75$-quantile that you computed is at the right edge of the shaded region, and thus the shaded region has area exactly $0.75$. To produce the shaded region use the `fill_between` method of `plt` (see the [docs](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.fill_between.html)!), and be sure to pass in the parameter `alpha=0.2` to change the opacity.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



## Expectations, variances, and standard deviations

### Description

SciPy implements methods to compute all the summary statistics of random variables that we have learned in class so far. For example, let's compute the mean, variance, and standard deviation of a random variable $X \sim Beta(3,7)$:

In [ ]:
X = sp.stats.beta(a=3, b=7)
print(X.mean())
print(X.var())
print(X.std())

Notice that SciPy uses `a` and `b` for the parameters $\alpha$ and $\beta$ of a beta distribution (see the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html)!).

### Problem 4 --- Means of beta variables

Your only goal in this problem is to produce the following plot:

<center>
<img src="https://github.com/jmyers7/stats-book-materials/blob/623c07132ab303636b984b4d3a2a1dfbe07aaa38/img/beta-mean.png?raw=true" width="600" align="center">
</center>

To produce the vertical line at the mean, use the `axvline` method of `plt` (see the [docs](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.axvline.html)!). Be sure to use the color `orange` for the vertical line. Notice also the legend in the upper right-hand corner!

In [ ]:
# ENTER YOUR CODE IN THIS CELL



## Sampling and empirical distributions

### Description

SciPy implements the `rvs` method to produce samples from a given distribution. For example, suppose that you wanted to produce $200$ random samples drawn from a $Pois(7)$ distribution. Then we would write:

In [ ]:
X = sp.stats.poisson(mu=7)
sample = X.rvs(size=200, random_state=42)
sample

As you can see, the `sample` object is a NumPy array containing 200 samples from a random variable $X\sim Pois(7)$. We passed in the parameter `random_seed=42` to set the seed for the random number generator for reproducible results (this is _very_ important!).

Now that we have a dataset on hand consisting of the observations in `sample`, we may use what we learned in the [third programming assignment](https://github.com/jmyers7/stats-book-materials/blob/main/programming-assignments/assignment_03.ipynb) to produce empirical distributions. For example, suppose that we wanted to compute the empirical PMF of the data in `sample`. First, we would write:

In [ ]:
support, counts = np.unique(ar=sample, return_counts=True)
support, counts

In this code block, we called the `unique` method in NumPy on the array `sample` to obtain the number of times each $x$-value appears in `sample`. The first array `support` contains the observed $x$-values, while the second array `counts` contains the counts. So, for example, we see that $x=2$ appears four times in `sample`.

Now, we need to convert these raw counts into probabilities. According to the _definition_ of an empirical distribution, we may do this by dividing element-wise `counts` by the total size of the dataset. After this, we create a Pandas Series containing the probabilities as the data column and the $x$-values as the indices:

In [ ]:
empirical_probs = counts / 200
empirical_pmf = pd.Series(data=empirical_probs, index=support, name='empirical')
empirical_pmf

Let's now compare this empirical PMF to the _true_ PMF of our random variable $X\sim Pois(7)$. First, let's generate the true PMF over the support of the empirical PMF:

In [ ]:
true_probs = X.pmf(support)
true_pmf = pd.Series(data=true_probs, index=support, name='true')
true_pmf

To facilitate easy comparison, let's combine our two Pandas Series into a single DataFrame called `df`:

In [ ]:
df = pd.concat(objs=[empirical_pmf, true_pmf], axis=1)
df

The probabilities in the two columns _should_ be relatively close to each other. To visualize them, let's produce a probability histogram combining both PMFs:

In [ ]:
df.plot(kind='bar', rot=0)
plt.xlabel('$x$')
plt.ylabel('probability')
plt.title('comparison of true and empirical distributions for $X\sim Pois(7)$')
plt.show()

Not such a bad approximation, right?

### Problem 5 -- Random samples from normal variables

It's your turn to produce random samples in SciPy and compare empirical and true distributions.

First, generate a random sample of size $250$ from a standard normal variable $Z \sim N(0,1^2)$. Name your random variable `Z` and save the sample into the variable `sample`. Be sure to pass in the parameter `random_state=42` for reproducibility:

In [ ]:
# ENTER YOUR CODE IN THIS CELL



In [ ]:
# Autograder cell. Do NOT alter or delete this cell.


Now, your goal is to compare the empirical distribution to the true distribution by producing this plot:

<center>
<img src="https://github.com/jmyers7/stats-book-materials/blob/1bf8d2592ec7f62327b16e1f5786a4357a82a768/img/norm-emp.png?raw=true" width="600" align="center">
</center>

Here are some hints:

* The random sample is drawn from a _continuous_ distribution, so you will produce a "binned" histogram for the empirical distribution. You learned about these in the [previous programming assignment](https://github.com/jmyers7/stats-book-materials/blob/main/programming-assignments/assignment_03.ipynb).

* Use $20$ bins for your histogram and set the edge color of the rectangles to black by passing in the parameter `ec='black'`. Set the opacity by passing in `alpha=0.2`.

* Plot the true density of $Z$ over the interval $[-3,3]$.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



We also learned in the [previous programming assignment](https://github.com/jmyers7/stats-book-materials/blob/main/programming-assignments/assignment_03.ipynb) how to compute empirical (or sample) means, variances, and standard deviations. Remember, these are supposed to be approximations to the _true_ means, variances, and standard deviations.

Well, we know that the mean of $Z\sim N(0,1^2)$ is $0$, while its variance and standard deviation are both $1$. In the next code block, compute the empirical mean of `sample`. Save your answer into the variable `mean`, and be sure to print it out to make sure it is close to $0$.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



In [ ]:
# Autograder cell. Do NOT alter or delete this cell.


Now, compute the empirical standard deviation of `sample`. Save your answer into the variable `sd`, and make sure to print it out to check that it is close to $1$.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



In [ ]:
# Autograder cell. Do NOT alter or delete this cell.
